# Trabajo Práctico: Agente para automatizar la búsqueda de repuestos

- **Curso:** DUIA - 2025, Módulo 6
- **Integrantes:** David Burckhardt, Martin Vazquez Arispe, Martin Caballero.
- **Objetivo:** Implementar un sistema inteligente que automatice la búsqueda, ranking y pedido de repuestos para una empresa distribuidora.

---
##  Índice del Notebook

1. **Consigna del trabajo**
2. **Configuracion de API Keys**

---

## 1. Consigna: agente(s) para automatizar la búsqueda de repuestos
- Dada una solicitud de repuestos específicos para una empresa distribuidora, un
agente debe identificar las especificaciones de dichos repuestos (según un
catálogo), a fin de poder buscarlos.
- El agente busca en primer lugar en el inventario de la empresa, y en en caso de
no encontrarlos (puede ser que encuentre solo algunos de ellos), debe consultar
en catálogos de proveedores.
- El sistema extrae información de las opciones encontradas, y genera un ranking
de alternativas, priorizando: 
    - Repuestos internos (si están disponibles).
    - Proveedores externos según criterios de optimización (por ej. costo-beneficio).
- Para repuestos internos: 
    - Se genera una orden de retiro del inventario y se notifica al almacén para su preparación. 
- Para repuestos externos: 
    - se envía un email automatizado al proveedor seleccionado para formalizar el pedido.

- Finalmente, se agenda la fecha estimada de entrega y detalles del pedido en el
sistema de seguimiento.
- Pueden incluirse pasos de "human in the loop" para verificar resultados antes de
tomar acciones

## 2. Configuración de API Keys y variables de entorno

Cargamos la `GROQ_API_KEY` desde el archivo `.env` e inicializamos el cliente LLM.

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
GROQ_API_KEY=tu_clave_aqui
```

In [6]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# Cargar variables de entorno
load_dotenv()

# Verificar que la API key está configurada
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY no encontrada en .env")

# Inicializar el LLM de Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    api_key=api_key
)

print("LLM de Groq inicializado correctamente")
print(f"   Modelo: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")


LLM de Groq inicializado correctamente
   Modelo: llama-3.3-70b-versatile
   Temperature: 0.1


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

with open('prompts/system_prompt.txt', 'r', encoding='utf-8') as f:
    SYSTEM_PROMPT = f.read()

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

# Create the chain
chain = prompt | llm


In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Optional
from langgraph.checkpoint.memory import MemorySaver
import re


#Definimos el esquema mejorado
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    codigo_repuesto: Optional[str]  # Código del repuesto (R-XXXX)
    info_completa: bool  # Si tenemos toda la información necesaria

#Defino funcion del nodo
def nodo_llm(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = chain.invoke({"messages": messages})
    
    # Extraer código de repuesto si está presente en los mensajes
    codigo_repuesto = state.get("codigo_repuesto")
    
    # Buscar patrón R-XXXX en todos los mensajes del usuario
    for msg in messages:
        if isinstance(msg, HumanMessage):
            # Buscar patrón R-XXXX (ej: R-0001, R-0123)
            match = re.search(r'R-\d{4}', msg.content, re.IGNORECASE)
            if match:
                codigo_repuesto = match.group(0).upper()
                break
    
    # Determinar si tenemos información completa
    info_completa = codigo_repuesto is not None
    
    return {
        "messages": [response],
        "codigo_repuesto": codigo_repuesto,
        "info_completa": info_completa
    }

#Defino la funcion para decidir si continuamos o terminamos
def tiene_info_suficiente(state: AgentState) -> str:
    # Si tenemos el código del repuesto, terminamos
    if state.get("info_completa", False) and state.get("codigo_repuesto"):
        print(f"\n✅ Código de repuesto identificado: {state['codigo_repuesto']}")
        return "end"
    
    # Si no, continuamos la conversación
    return "continue"

memory = MemorySaver()

#Defino el grafo
graph_builder = StateGraph(AgentState)

#Agrego el nodo al grafo
graph_builder.add_node("llm_node", nodo_llm)

#Conecto los nodos (Solo uno por ahora)
graph_builder.add_edge(START, "llm_node")
graph_builder.add_conditional_edges(
    "llm_node",
    tiene_info_suficiente,
    {
        "continue": "llm_node",  # Vuelve a ejecutar llm_node
        "end": END
    }
)

graph = graph_builder.compile(
    checkpointer=memory, 
    interrupt_after=["llm_node"]
)

In [9]:
#Defino la funcion del agente
def iniciar_agente(mensaje_usuario: str):
    config = {"configurable": {"thread_id": "1"}}
    
    # Estado inicial con todos los campos
    estado_inicial = {
        "messages": [HumanMessage(content=mensaje_usuario)],
        "codigo_repuesto": None,
        "info_completa": False
    }
    
    result = graph.invoke(estado_inicial, config)
    
    # Loop de conversación
    while True:
        # Mostrar respuesta del agente
        print(f"\n🤖 Agente: {result['messages'][-1].content}")
        
        # Verificar si llegamos al final (tenemos el código)
        if result.get("info_completa", False):
            print(f"\n{'='*60}")
            print(f"📋 Información recopilada:")
            print(f"   Código del repuesto: {result['codigo_repuesto']}")
            print(f"{'='*60}")
            break
        
        # Pedir nuevo mensaje al usuario
        nuevo_mensaje = input("\n👤 Tú: ")
        
        if nuevo_mensaje.lower() in ["salir", "exit", "quit"]:
            print("\n👋 Conversación terminada")
            break
        
        # Continuar la conversación
        nuevo_estado = {"messages": [HumanMessage(content=nuevo_mensaje)]}
        result = graph.invoke(nuevo_estado, config)
    
    return result

## Ejemplos de uso del agente

**Ejemplos de códigos de repuestos en la base de datos:**
- R-0001: Rodamiento rígido de bolas 6204 2RS
- R-0002: Filtro de aceite motor diésel
- R-0005: Bomba centrífuga 3 HP
- R-0010: Manómetro glicerina 0-16 bar

**Categorías disponibles:**
- RODAMIENTO, FILTRO, CORREA, SENSOR, BOMBA, ELECTRICO, NEUMATICA, MECANICO, INSTRUMENTO


In [10]:
#main
print("="*60)
print("🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS")
print("="*60)
print("\nBienvenido al sistema de búsqueda de repuestos.")
print("El agente te ayudará a encontrar el repuesto que necesitas.")
print("\nPuedes escribir 'salir' en cualquier momento para terminar.\n")
print("-"*60)

mensaje_usuario = input("\n👤 Tú: ")
resultado = iniciar_agente(mensaje_usuario)


🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS

Bienvenido al sistema de búsqueda de repuestos.
El agente te ayudará a encontrar el repuesto que necesitas.

Puedes escribir 'salir' en cualquier momento para terminar.

------------------------------------------------------------

🤖 Agente: ¡Hola! Me alegra poder ayudarte a encontrar el repuesto que necesitas. Para empezar, ¿podrías proporcionarme el código del repuesto que estás buscando? El código debe tener el formato R-XXXX (por ejemplo, R-0001). Si no lo conoces, también puedo ayudarte con la descripción o la categoría del repuesto. ¿Qué información tienes sobre el repuesto que necesitas? 

Algunas categorías que tenemos son:
* RODAMIENTO
* FILTRO
* CORREA
* SENSOR
* BOMBA
* ELECTRICO
* NEUMATICA
* MECANICO
* INSTRUMENTO

¿Puedes darme algún detalle para empezar la búsqueda?

✅ Código de repuesto identificado: R-0001

🤖 Agente: ¡Excelente! Gracias por proporcionarme el código del repuesto: **R-0001**.

Antes de proceder a la búsqueda, solo quie